In [1]:
%matplotlib inline
import openpyxl
#from astropy.table import Table
from astropy.table import vstack, Table
import astropy.io.fits as fits
import matplotlib.pyplot as plt
import numpy as np
import egs_morph
import time
import os
import astropy.stats as stats

import warnings
warnings.filterwarnings("ignore")

In [2]:


egs_path = '/home/hqzhang/Data_EGS/EGS_tot_v1/science_images/'  #22103
psf_path = "psf/"
type_g = 'disk'
fig_path = '/home/wangjh/Mywork/EGS/EGS_figure/%s/'%type_g
result_path = '/home/wangjh/Mywork/EGS/EGS_result/%s/'%type_g
if not os.path.exists(result_path):
    os.makedirs(result_path)
psf_v = fits.open(psf_path + "epsf_v.fits")[0].data
psf_i = fits.open(psf_path + "epsf_i.fits")[0].data
psf_j = fits.open(psf_path + "epsf_j.fits")[0].data
psf_h = fits.open(psf_path + "epsf_h.fits")[0].data

ws = openpyxl.load_workbook('egs_type/egs_%s.xlsx'%type_g)['Sheet1']

begin = 3401
#end = 2353 #sph
end = 6815 #disk
#end = 814 #irr

In [3]:
failed_list = []
Rp_5_list = []
no_list = []

In [4]:
def psf_final(band):
    if band == 'v':
        psf = psf_v
    elif band == 'i':
        psf = psf_i
    elif band == 'j':
        psf = psf_j
    elif band == 'h':
        psf = psf_h
    return psf

In [5]:
bands = ['v','i','j','h']
table = Table(names=('id','band','redshift','mass','sfr','p_sph','p_disk','p_irr','gal_ser_n_h','Sersic_n','flag_n','snr',\
                       'r20(kpc)','r50(kpc)','re(kpc)','r80(kpc)','r_petro','ellip','C','A','S','Gini','M20','M20_c'),\
                       dtype=('S','S','f2','f2','f4','f2','f2','f2','f2','f2','I',\
                              'f4','f4','f4','f4','f4','f4','f4','f4','f4','f4','f4','f4','f4'))
for i in range(begin,end+1):
    start = time.time()
    id_name = ws.cell(row=i, column=1).value
    print('%s is start _%d'%(id_name,i))
    redshift = ws.cell(row=i, column=6).value    
    mass = ws.cell(row=i, column=7).value
    sfr = ws.cell(row=i, column=8).value
    gilfit_n = ws.cell(row=i, column=9).value
    type_sph = ws.cell(row=i, column=10).value
    type_disk = ws.cell(row=i, column=11).value
    type_irr = ws.cell(row=i, column=12).value
    
    
    img_path = egs_path + '%s/'%id_name
    if os.path.exists(img_path):
        mask = fits.open(egs_path + '%s/%s_mask.fits'%(id_name,id_name))[0].data
        for band in bands:
            image_data = fits.open(egs_path + '%s/%s_%s.fits'%(id_name,id_name,band))[0].data

            psf = psf_final(band)
            image_morph = egs_morph.mymorph(id_name,image_data,psf,mask,redshift ,mass,sfr,band,type_sph,type_disk,type_irr,gilfit_n) 
            bbb = image_morph.save_fig
            if bbb!= 0:        
                bbb.savefig(fig_path + '%s_%s.png'%(id_name,band))
            image_tabel = image_morph.image_table
            table = vstack([table,image_tabel]) 
            if image_morph.worked == 0:
                failed_list.append('%d_%s'%(id_name,band))
            else:
                if image_morph.Rp5 == 0:
                    Rp_5_list.append('%d_%s'%(id_name,band))
        print('%s is done'%id_name,'Time: %g s.' % (time.time() - start))
    else:       
        no_list.append('%d'%id_name)
        print('%s is not exsit'%id_name)
        pass

15864 is start _3401
15864 is done Time: 2.96138 s.
15865 is start _3402
15865 is done Time: 3.11336 s.
15866 is start _3403
15866 is done Time: 3.02285 s.
15871 is start _3404
15871 is done Time: 2.95741 s.
15887 is start _3405
15887 is done Time: 2.34247 s.
15888 is start _3406
15888 is done Time: 2.2757 s.
15894 is start _3407
15894 is done Time: 2.56745 s.
15899 is start _3408
15899 is done Time: 2.40326 s.
15900 is start _3409
15900 is done Time: 2.59597 s.
15903 is start _3410
15903 is not exsit
15906 is start _3411
15906 is done Time: 2.85092 s.
15908 is start _3412
15908 is done Time: 2.90098 s.
15919 is start _3413
15919 is not exsit
15935 is start _3414
15935 is done Time: 3.15241 s.
15946 is start _3415
15946 is done Time: 3.10417 s.
15947 is start _3416
15947 is done Time: 2.99113 s.
15951 is start _3417
15951 is done Time: 3.13624 s.
15956 is start _3418
15956 is done Time: 2.90586 s.
15960 is start _3419
15960 is done Time: 3.02756 s.
15966 is start _3420
15966 is done Ti

In [6]:
#table.show_in_notebook()

In [7]:
table.write(result_path +'%s2_result.fits'%type_g, overwrite=True)

In [8]:
with open(result_path + "failed_%s_%d-%d.txt"%(type_g,begin,end),'w') as failed_l:
    [failed_l.write(str(item)+'\n') for item in failed_list]
    failed_l.close    

In [9]:
with open(result_path + "Rp5_%s_%d-%d.txt"%(type_g,begin,end),'w') as Rp5_l:
    [Rp5_l.write(str(item)+'\n') for item in Rp_5_list]
    Rp5_l.close    

In [10]:
with open(result_path + "no_%s_%d-%d.txt"%(type_g,begin,end),'w') as no_l:
    [no_l.write(str(item)+'\n') for item in no_list]
    no_l.close    

In [11]:
print('done')

done
